In [5]:
!pip install numpy librosa sounddevice pyaudio
!pip install torch torchaudio scikit-learn matplotlib
import sounddevice as sd
import numpy as np
import librosa 
import torch

  Using cached torch-2.9.1-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached torchaudio-2.9.1-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.9.1-cp312-cp312-win_amd64.whl (110.9 MB)
Using cached torchaudio-2.9.1-cp312-cp312-win_amd64.whl (665 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [6]:
import torch
import torch.nn as nn

class LSTMEmotion(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(40, 128, batch_first=True)
        self.fc = nn.Linear(128, 8)

    def forward(self,x):
        x = x.unsqueeze(1)
        _, (h,_) = self.lstm(x)
        return self.fc(h[-1])


In [9]:
model=LSTMEmotion()
model.load_state_dict(torch.load("model.pth"))
model.eval()

LSTMEmotion(
  (lstm): LSTM(40, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=8, bias=True)
)

In [11]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(['angry','calm','disgust','fearful','happy','neutral','sad','surprised'])


LabelEncoder()

In [ ]:
def record(seconds=5):
    print("recording....")
    audio=sd.rec(int(seconds*22050),samplerate=22050,channels=1)
    sd.wait()
    return audio.flatten()

while True:
    audio=record()
    mfcc=librosa.feature.mfcc(y=audio,sr=22050,n_mfcc=40)
    mfcc=np.mean(mfcc.T,axis=0)

    inp=torch.tensor(mfcc,dtype=torch.float32)
    inp=inp.unsqueeze(0)
    out=model(inp)
    emotion=torch.argmax(out).item()
    print("emotion:",encoder.classes_[emotion])


recording....


C:\Users\cheda\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\cheda\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\cheda\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


emotion: fearful
recording....
emotion: disgust
recording....
emotion: angry
recording....
emotion: sad
recording....
emotion: fearful
recording....
emotion: disgust
recording....
emotion: sad
recording....
emotion: sad
recording....
emotion: sad
recording....
emotion: sad
recording....
emotion: sad
recording....
emotion: fearful
recording....
emotion: fearful
recording....
